# KVLCC2 Ikeda at various frequencies

# Purpose
Will the viscous damping predicted with Ikeda change with frequency?

# Methodology
Make a variation around the natural frequency

## Results
Describe and comment the most important results.

# Suggested next steps
State suggested next steps, based on results obtained in this notebook.

# Setup

In [ ]:
# %load imports.py
"""
These is the standard setup for the notebooks.
"""

%matplotlib inline
%load_ext autoreload
%autoreload 2

from jupyterthemes import jtplot
jtplot.style(theme='onedork', context='notebook', ticks=True, grid=False)

import pandas as pd
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option("display.max_columns", None)
import numpy as np
import os
import matplotlib.pyplot as plt
from collections import OrderedDict
#plt.style.use('paper')

#import data
import copy
from mdldb.run import Run

from sklearn.pipeline import Pipeline
from rolldecayestimators.transformers import CutTransformer, LowpassFilterDerivatorTransformer, ScaleFactorTransformer, OffsetTransformer
from rolldecayestimators.direct_estimator_cubic import EstimatorQuadraticB, EstimatorCubic
from rolldecayestimators.ikeda_estimator import IkedaQuadraticEstimator
import rolldecayestimators.equations as equations
import rolldecayestimators.lambdas as lambdas
from rolldecayestimators.substitute_dynamic_symbols import lambdify
import rolldecayestimators.symbols as symbols
import sympy as sp

from sympy.physics.vector.printing import vpprint, vlatex
from IPython.display import display, Math, Latex

from sklearn.metrics import r2_score
from src.data import database
from mdldb import tables


In [ ]:
import pyscores2
import pyscores2.runScores2
import pyscores2.xml_hydrostatics
from pyscores2.output import OutputFile
from rolldecayestimators.ikeda import Ikeda, IkedaR

from rolldecayestimators.simplified_ikeda_class import SimplifiedIkeda, SimplifiedIkedaABS
from rolldecayestimators.simplified_ikeda import limits_kawahara
from pyscores2.runScores2 import Calculation
from pyscores2.indata import Indata
import joblib
from scipy.optimize import least_squares
from reports.paper_writing import save_fig

In [ ]:
db = database.get_db()

In [ ]:
sql = """
SELECT * from run
INNER JOIN loading_conditions
ON (run.loading_condition_id = loading_conditions.id)
INNER JOIN models
ON (run.model_number = models.model_number)
INNER JOIN ships
ON (run.ship_name = ships.name)
WHERE run.model_number='M5057-01-A' and run.test_type='roll decay' and run.project_number=40178362;
"""
df_rolldecays = pd.read_sql(sql=sql, con=db.engine)
df_rolldecays=df_rolldecays.loc[:,~df_rolldecays.columns.duplicated()]
df_rolldecays.set_index('id', inplace=True)

df_rolldecays['ship_speed'].fillna(0, inplace=True)


In [ ]:
df_rolldecays.head()

In [ ]:
def calculate_ikeda(ikeda):

    output = pd.DataFrame()
    output['B_44_hat']   = ikeda.calculate_B44()
    output['B_W0_hat']   = ikeda.calculate_B_W0()
    output['B_W_hat']    = ikeda.calculate_B_W()
    output['B_F_hat']    = ikeda.calculate_B_F()
    output['B_E_hat']    = ikeda.calculate_B_E()
    output['B_BK_hat']   = ikeda.calculate_B_BK()
    output['B_L_hat']    = ikeda.calculate_B_L()
    output['Bw_div_Bw0'] = ikeda.calculate_Bw_div_Bw0()
    return output

In [ ]:
#df_rolldecays=df_rolldecays.loc[[21337,21338,21340,]].copy()
df_rolldecays=df_rolldecays.loc[[21338,21340,]].copy()

In [ ]:
resources={
    21338 : {
        'scores_indata_path':'../models/KVLCC2_speed.IN',
        'scores_outdata_path':'../data/interim/KVLCC2_speed.out',
        'roll_decay_model':'../models/KVLCC2_0_speed.pkl',
    },
    21340 : {
        'scores_indata_path':'../models/KVLCC2_speed.IN',
        'scores_outdata_path':'../data/interim/KVLCC2_speed.out',
        'roll_decay_model':'../models/KVLCC2_speed.pkl',
    }
}

In [ ]:
def unhat(df_ikeda, volume, beam, g, rho):
    for key,data in df_ikeda.items():
        if '_hat' in key:
            new_key = key.replace('_hat','')
            df_ikeda[new_key] = lambdas.B_from_hat_lambda(B_44_hat=data, Disp=volume, beam=beam, g=g, rho=rho)
            
    return df_ikeda  

In [ ]:
g=9.81
rho=1000

phi_a = np.deg2rad(10)
results = pd.DataFrame()

for id, row in df_rolldecays.iterrows():
    run = db.session.query(Run).get(int(row.name))
    run = database.load_run(run, save_as_example=False, prefer_hdf5=True)
    scale_factor = run.model.scale_factor
    
    resource = resources[id]
    
    ## Load ScoresII results
    indata = Indata()
    indata.open(indataPath='../models/KVLCC2_speed.IN')
    output_file = OutputFile(filePath='../data/interim/KVLCC2_speed.out')
    
    ## Compare with model test
    model = joblib.load(resource['roll_decay_model'])
    estimator = model['estimator']
    
    ## Non. Lin. linear equivalent damping
    GM = run.loading_condition.gm/scale_factor
    volume = run.loading_condition.Volume/(scale_factor**3)
    GM = run.loading_condition.gm/scale_factor
    beam = run.ship.beam/scale_factor

    meta_data = {
        'Volume':volume,
        'GM':GM,
        'rho':rho,
        'g':g,
    }
    parameters = estimator.result_for_database(meta_data = meta_data)

    ## Run Ikeda
    N=99
    ws_factor = np.linspace(0.5,1.5,N)
    ws = parameters['omega0']*ws_factor
    
    scale_factor=run.model.scale_factor
    V = row.ship_speed*1.852/3.6/np.sqrt(scale_factor)
    Vs = V*np.ones(len(ws))
    phi_as = phi_a*np.ones(len(ws))
    
    if not run.ship.BKL:
        BKL=0
    else:
        BKL=run.ship.BKL/scale_factor
    
    if not run.ship.BKB:
        BKB = 0
    else:
        BKB=run.ship.BKB/scale_factor
   
    
    BKL_ = BKL*np.ones(N)
    BKB_ = BKB*np.ones(N)
       
    ikeda = Ikeda.load_scoresII(V=Vs, w=ws, fi_a=phi_as, indata=indata, output_file=output_file, 
                                scale_factor=scale_factor, BKL=BKL_, BKB=BKB_)
    
    R = 0.15*run.ship.beam/scale_factor  # Just guessing...
    ikeda.R = R
    
    df_ikeda = calculate_ikeda(ikeda=ikeda)
    df_ikeda['w']=ws
    df_ikeda['ws_factor']=ws_factor
    df_ikeda['phi_a']=phi_as
    df_ikeda.set_index('ws_factor',inplace=True)
    df_ikeda['id']=id
    df_ikeda = unhat(df_ikeda=df_ikeda, volume=volume, beam=beam, g=g, rho=rho)
    
    # assuming : phi = phi_a*sin(w*t)
    #phi1ds = phi_as*w*np.cos()
    
    #df_ikeda['B'] = (parameters['B_1'] + parameters['B_2']*np.abs(ws))*ws + parameters['B_3']*ws**3
    #df_ikeda['B']=df_ikeda['B'].astype(float)
    
    df_ikeda['B']=lambdas.B_e_lambda_cubic(B_1=parameters['B_1'], B_2=parameters['B_2'], B_3=parameters['B_3'], omega0=ws, phi_a=phi_as)
    df_ikeda['B']=df_ikeda['B'].astype(float)
    
    results=results.append(df_ikeda)
        

In [ ]:
parameters

In [ ]:
equations.B_e_equation_cubic

<a id='himeno'></a>

In [ ]:
for id,df_ikeda in results.groupby(by='id'):
    fig,ax=plt.subplots()
    interesting_ = ['B_W','B_L','B_F','B_E',]
      
    df_ikeda.plot.area(x='w', y=interesting_, ax=ax)
    
    #df_ikeda.plot(x='w', y='B', ax=ax)
    df_ikeda.plot(x='w', y='B', label=r'$B_e$ roll decay', ax=ax)
    
    ax.legend()
    ax.set_xlabel(r'$ \omega $ [rad/s] (Roll frequency)')
    ax.set_ylabel(r'$ B_e $ [Nm/s]');
    
    s = df_rolldecays.loc[id]
    phi_a = np.rad2deg(df_ikeda['phi_a'].iloc[0])
    title='Ship speed: %0.1f [kts] $\phi_a$: %i [deg]' % (s.ship_speed,phi_a)
    ax.set_title(title)    
   


<a id='frequency'></a>

In [ ]:
for id,df_ikeda in results.groupby(by='id'):
    fig,ax=plt.subplots()
    interesting_ = ['B_L_hat','B_F_hat','B_E_hat',]
    df_ikeda_fraction = df_ikeda[interesting_].copy()
    df_ikeda_fraction/=df_ikeda_fraction.loc[1]
    
    df_ikeda_fraction.plot.area(y=interesting_, ax=ax)
        
    
    ax.legend()
    ax.set_xlabel(r'$ \frac{\omega}{\omega_0} $ [-] (Roll frequency)')
    ax.set_ylabel(r'$\frac{\hat{B_e}}{\hat{B_e^0}}$ [-]');
    
    s = df_rolldecays.loc[id]
    phi_a = np.rad2deg(df_ikeda['phi_a'].iloc[0])
    title='Ship speed: %0.1f [kts] $\phi_a$: %i [deg]' % (s.ship_speed,phi_a)
    ax.set_title(title)    
    
    figure_name = 'KVLCC2_B_visc_frequency%0.1f' % s.ship_speed
    save_fig(fig, name=figure_name)
    